In [1]:
# graph representation in Adjacency Matrix

class Graph:
    def __init__(self, num_vertices):
        self.graph = [[0]*num_vertices for _ in range(num_vertices)]
        self.num_vertices = num_vertices
    
    def add_edge(self, u, v, weight=1):
        self.graph[u][v] = weight
        self.graph[v][u] = weight # for undirected graph

In [6]:
class DisjointSet:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0]*n
    
    def find(self, u):
        if self.parent[u]!=u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]
    
    def union(self,u,v):
        root_u = self.find(u)
        root_v = self.find(v)
        if root_u != root_v:
            if self.rank[root_u] > self.rank[root_v]:
                self.parent[root_v] = root_u
            elif self.rank[root_u] < self.rank[root_v]:
                self.parent[root_u] = root_v
            else:
                self.parent[root_v] = root_u
                self.rank[root_u] += 1

In [8]:
# representation in Adjacency List:
from collections import defaultdict, deque
import heapq
class Graph:
    def __init__(self, num_vertices):
        self.graph = defaultdict(list)
        self.num_vertices = num_vertices
    
    def add_edge(self, u, v):
        self.graph[u].append(v)
        self.graph[v].append(u) # for undirected graph

    def dfs(self, start, visited = None):
        if visited is None:
            visited = set()
        visited.add(start)
        print(start, end=' ')
        for neighbor in self.graph[start]:
            if neighbor not in visited:
                self.dfs(neighbor, visited)
    
    def bfs(self, start):
        visited = set()
        queue = deque([start])
        visited.add(start)
        while queue:
            vertex = queue.popleft()
            print(vertex, end=' ')
            for neighbor in self.graph[vertex]:
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
    
    def topological_sort_util(self, v, visited, stack):
        visited[v] = True
        for neighbor in self.graph[v]:
            if not visited[neighbor]:
                self.topological_sort_util(neighbor, visited, stack)
        stack.insert(0,v)
    
    def topological_sort(self):
        visited = {vertex: False for vertex in self.graph}
        stack = []
        for vertex in self.graph:
            if not visited[vertex]:
                self.topological_sort_util(vertex, visited, stack)
        return stack
    
    def is_cyclic_util(self, v, visited, parent):
        visited[v] = True
        for neighbor in self.graph[v]:
            if not visited[neighbor]:
                if self.is_cyclic_util(neighbor, visited, v):
                    return True
            elif parent != neighbor:
                return True
        return False
    
    def is_cyclic(self):
        visited = [False]*self.num_vertices
        for vertex in range(self.num_vertices):
            if not visited[vertex]:
                if self.is_cyclic_util(vertex, visited, -1):
                    return True
        return False
    
    # Prims algo for Minimum spanning tree

    def prim(self, start):
        mst = []
        visited = set([start])
        edges = [(cost, start, to) for to, cost in self.graph[start].items()]
        heapq.heapify(edges)

        while edges:
            cost, frm, to = heapq.heappushpop(edges)
            if to not in visited:
                visited.add(to)
                mst.append((frm, to, cost))

                for to_next, cost in self.graph[to].items():
                    if to_next not in visited:
                        heapq.heappush(edges, (cost, to, to_next))
        return mst
    
    #krushkal's Algo

    def kruskal(self, graph):
        mst = []
        edges = sorted(graph, key=lambda item:item[2])
        ds = DisjointSet(self.num_vertices)

        for edge in edges:
            u,v, weight = edge
            if ds.find(u) != ds.find(v):
                ds.union(u,v)
                mst.append(edge)
        return mst

g = Graph(5)
g.add_edge(0, 1)
g.add_edge(0, 4)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(1, 4)
g.add_edge(2, 3)
g.add_edge(3, 4)

print("DFS Traversal:")
g.dfs(0)
print("\nBFS Traversal:")
g.bfs(0)
print("\nTopological Sort:")
print(g.topological_sort())
print("\nCycle Detection (Undirected):")
print(g.is_cyclic())

DFS Traversal:
0 1 2 3 4 
BFS Traversal:
0 1 4 2 3 
Topological Sort:
[0, 1, 2, 3, 4]

Cycle Detection (Undirected):
True


### Rotten oranges

In [1]:
from collections import deque

def rottenoranges(grid):
    rows, col = len(grid), len(grid[0])
    queue = deque()
    fresh_oranges = 0

    for r in range(rows):
        for c in range(col):
            if grid[r][c] == 2:
                queue.append((r,c))
            elif grid[r][c] == 1:
                fresh_oranges += 1
    print(queue)
    
    directions = [(1,0), (-1,0), (0,1), (0,-1)]
    minutes_passed = 0


    while queue and fresh_oranges > 0:
        minutes_passed += 1
        for _ in range(len(queue)):
            x, y = queue.popleft()
            print(queue)
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                print(f"dx: {dx} and dy: {dy}")
                if 0 <= nx < rows and 0 <= ny < col and grid[nx][ny] == 1:
                    grid[nx][ny] = 2
                    fresh_oranges -= 1
                    queue.append((nx,ny))
                    print(grid)
                    print(queue)
                    print("---")
    
    return minutes_passed if fresh_oranges == 0 else -1

### Surrounded Regions

In [2]:
def solve(board):
    if not board or not board[0]:
        return 
    
    rows, cols = len(board), len(board[0])

    def dfs(r, c):
        if r < 0 or c < 0 or r >= rows or c >= cols or board[r][c] != 'O':
            return
        board[r][c] = "T"
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for dr, dc in directions:
            dfs(r + dr, c + dc)
    
    for r in range(rows):
        for c in [0, cols - 1]:
            if board[r][c] == "O":
                dfs(r, c)
    for r in [0, rows - 1]:
        for c in range(cols):
            if board[r][c] == "O":
                dfs(r, c)
    
    for r in range(rows):
        for c in range(cols):
            if board[r][c] == "T":
                board[r][c] == "O"
            elif board[r][c] == "O":
                board [r][c] == "X"

### number of islands

In [3]:
def solve(grid):
    if not grid or not grid[0]:
        return
    
    rows, cols = len(grid), len(grid[0])
    def dfs(r, c):
        if r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] == '0':
            return
        grid[r][c] = '1'
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        for dr, dc in directions:
            dfs(r + dr, c + dc)
    
    number_of_islands = 0

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                number_of_islands += 1
                dfs(r, c)
    return number_of_islands

In [4]:
def solve(grid):
    from collections import deque

    if not grid or not grid[0]:
        return
    
    rows, cols = len(grid), len(grid[0])
    island_count = 0

    def bfs(r, c):
        q = deque([(r, c)])
        grid[r][c] = '0'
        while q:
            x, y = q.popleft()
            directions = [(1, 0), (-1, 0), (0, 1), (0. -1)]
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] == '1':
                    grid[nx][ny] == '0'
                    q.append((nx, ny))
        
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                island_count += 1
                bfs(r, c)
    return island_count

### Cloning Graph

In [5]:
class Node:
    def __init__(self, val=0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

In [10]:
def cloneGraphs(node):
    if not node:
        return None
    
    visited = {}

    def dfs(node):
        if node in visited:
            return visited[node]
        clone = Node(node.val)
        visited[node] = clone

        for neighbor in node.neighbors:
            clone.neighbors.append(dfs(neighbor))

        return clone
    return dfs(node)

In [11]:
node1 = Node(1)
node2 = Node(2)
node3 = Node(3)
node4 = Node(4)
node1.neighbors = [node2, node4]
node2.neighbors = [node1, node3]
node3.neighbors = [node2, node4]
node4.neighbors = [node1, node3]

cloned_graph = cloneGraph(node1)

In [13]:
def cloneGraph(node):
    from collections import deque
    if not node:
        return None
    
    visited = {}
    q = deque()
    visited[node] = Node(node.val)
    while q:
        n = q.popleft()
        for neighbor in n.neighbors:
            if neighbor not in visited:
                visited[neighbor] = Node(neighbor.val)
                q.append(neighbor)
            visited[n].neighbors.append(visited[neighbor])
    return visited[node]


In [8]:
node1 = Node(1)
node2 = Node(2)
node3 = Node(3)
node4 = Node(4)
node1.neighbors = [node2, node4]
node2.neighbors = [node1, node3]
node3.neighbors = [node2, node4]
node4.neighbors = [node1, node3]

cloned_graph = cloneGraph(node1)

### Evaluate Division

In [16]:
from collections import defaultdict

def calcEquation(equations, values, queries):
    graph = defaultdict(dict)

    for (a, b), value in zip(equations, values):
        graph[a][b] = value
        graph[b][a] = 1/value
    
    def dfs(start, end, visited):
        if start not in graph or end not in graph:
            return -1.0
        if start == end:
            return 1.0
        visited.add(start)
        for neighbor, value in graph[start].items():
            if neighbor not in visited:
                result = dfs(neighbor, end, visited)
                if result != -1.0:
                    return result * value
        return -1.0
    
    results = []
    for a, b in queries:
        if a == b and a in graph:
            results.append(1.0)
        else:
            results.append(dfs(a, b, set()))
    
    return results

In [17]:
equations = [["a", "b"], ["b", "c"]]
values = [2.0, 3.0]
queries = [["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], ["x", "x"]]

print(calcEquation(equations, values, queries))  # Output: [6.0, 0.5, -1.0, 1.0, -1.0]


[6.0, 0.5, -1.0, 1.0, -1.0]
